In [ ]:
def classify(X, exemplar_sets):
    mean_of_exemplars=[]
    for i in range(len(exemplar_sets)):
        #calculate mean of each exemplar set using feature map
        #mean_of_exemplars.append(mean)
    #calcuate feature representation of X using feature map function
    #exemplar set closest to X feature map is chosen
    return 0